# The Impact of Transportation Methods on Air Quality in Big Cities

**Objective**: To analyze the relationship between transportation habits and air quality in urban areas to promote sustainable transportation options.

**Target Variable**: Air Quality (from metric_subcat_label)

**Relevant Independent Variables** (from metric_item_label):
- Public Transportation Use, 
- Drives Alone to Work, 
- Walking to Work, 
- Riding Bike to Work (from metric_item_label)


## 0. Requirements

In [32]:
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns



## 1. Motivation and Dataset Exploration
- What was your goal for the end user's experience?

In [ ]:
# dataset: https://bigcitieshealthdata.org/
data = pd.read_csv('BigCitiesHealth.csv')
data.head()

C:\Users\Lau\AppData\Local\Temp\ipykernel_3304\878469504.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('BigCitiesHealth.csv')


,metric_item_label,metric_cat_label,metric_subcat_label,metric_item_label_subtitle,metric_cat_item_yaxis_label,metric_source_desc_label_fn,metric_source_desc_label_url_fn,geo_label_city,geo_label_state,geo_label_citystate,...,value_90_ci_low,value_90_ci_high,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,strata_race_label,strata_sex_label,strata_race_sex_label
0,COVID-19 Deaths,Infectious Diseases,Respiratory Infection,"Deaths from COVID-19 (per 100,000 population, ...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,San Antonio,TX,"San Antonio, TX",...,NaN,NaN,South,Less poor cities (<20% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),White,Male,White Male
1,COVID-19 Deaths,Infectious Diseases,Respiratory Infection,"Deaths from COVID-19 (per 100,000 population, ...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Houston,TX,"Houston, TX",...,NaN,NaN,South,Poorest cities (20%+ poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),White,Male,White Male
2,COVID-19 Deaths,Infectious Diseases,Respiratory Infection,"Deaths from COVID-19 (per 100,000 population, ...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Fort Worth,TX,"Fort Worth, TX",...,NaN,NaN,South,Less poor cities (<20% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),White,Male,White Male
3,COVID-19 Deaths,Infectious Diseases,Respiratory Infection,"Deaths from COVID-19 (per 100,000 population, ...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,El Paso,TX,"El Paso, TX",...,NaN,NaN,South,Less poor cities (<20% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),White,Male,White Male
4,COVID-19 Deaths,Infectious Diseases,Respiratory Infection,"Deaths from COVID-19 (per 100,000 population, ...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Dallas,TX,"Dallas, TX",...,NaN,NaN,South,Less poor cities (<20% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),White,Male,White Male


> **Brief Data Description**

The data we've chosen to work with is **The Big Cities Health Inventory (BCHI) data platform**.
> BCHI contains over 120 metrics across key public health categories, chosen for their impact on morbidity, mortality, and community well-being in the US. Metrics must meet substantive criteria relating to public health relevance, policy benchmarking, or demographic/socioeconomic disparity highlighting, as well as coverage and standardization criteria. The platform prioritizes publicly available, city-level data, with information from 2010 to 2021. The variables we use in our analysis are:

**Target Variable**: "Air Quality"
- Calculated using Air Quality Index (AQI) published by the Environmental Protection Agency. The 'Air Quality' value is calculated as $\frac{\text{No. of days with AQI} > 50}{\text{No. of days recorded}}$ [p. 15 in data description]

**Independent Variables**
- **1**. "Walking to Work" and **2**. "Riding Bike to Work": Calculated using the U.S. Census Bureau’s American Community Survey (ACS) 5-year estimates and is calculated from table B08137. The numerator is those who reported walking or riding a bicycle to work respectively. The denominator is all workers aged 16+ years who did not work at home. [p. 16 in data description]
- **3**. "Public Transportation Use" and **4**. "Drives Alone to Work": Calculated using the U.S. Census Bureau’s American Community Survey (ACS) 5-year estimates. The value is calculated as $\frac{\text{No. of citizens who reported using public transport}}{\text{No. of workers ages 16+ that who did not work from home}}$ and similarily for "Drives Alone to Work". [p. 16 in data description]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170725 entries, 0 to 170724
Data columns (total 31 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   metric_item_label                170725 non-null  object 
 1   metric_cat_label                 170725 non-null  object 
 2   metric_subcat_label              170725 non-null  object 
 3   metric_item_label_subtitle       170725 non-null  object 
 4   metric_cat_item_yaxis_label      170725 non-null  object 
 5   metric_source_desc_label_fn      170725 non-null  object 
 6   metric_source_desc_label_url_fn  170725 non-null  object 
 7   geo_label_city                   170725 non-null  object 
 8   geo_label_state                  169516 non-null  object 
 9   geo_label_citystate              170725 non-null  object 
 10  geo_fips_code                    169516 non-null  float64
 11  value                            169957 non-null  float64
 12  da

In [ ]:
data.describe()

,geo_fips_code,value,date_label,value_95_ci_low,value_95_ci_high,value_90_ci_low,value_90_ci_high
count,1.695160e+05,169957.000000,170725.000000,110314.000000,110325.000000,0.0,0.0
mean,2.873797e+06,799.240387,2015.725102,446.619734,539.482406,NaN,NaN
std,1.697982e+06,4241.009061,3.385490,1747.515240,2039.646166,NaN,NaN
min,4.550000e+05,-1.052184,2010.000000,-148.037039,0.458320,NaN,NaN
25%,8.200000e+05,10.992568,2013.000000,9.400000,15.150000,NaN,NaN
50%,2.938000e+06,25.073544,2016.000000,20.757485,31.136362,NaN,NaN
75%,4.748000e+06,73.582871,2019.000000,91.258907,129.890367,NaN,NaN
max,5.553000e+06,99408.000000,2021.000000,27873.280170,29954.101410,NaN,NaN


We chose this particular dataset because looking for datasets from different cities that contain similar and complete data is a challenging task, but this dataset contained all the information we needed to do the analysis on air quality in cities. It has enough data to fins some interesting insights, this is, we have geographical data, we have time data, air quality, attributes such as means of transport that might impact on the air quality, among others. 

# 2. Basic stats

We only keep the attributes of interest for our analysis, the rest of the columns are dropped. 

In [ ]:
columns_to_keep = ['geo_label_city', 'geo_label_citystate', 'value', 'date_label', 'metric_item_label', 'metric_subcat_label']
data = data[columns_to_keep]

data['date_label'] = pd.to_datetime(data['date_label'], format='%Y', errors='coerce')

print(data.isna().sum())

geo_label_city           0
geo_label_citystate      0
value                  768
date_label               0
metric_item_label        0
metric_subcat_label      0
dtype: int64


### 2.1. Handling Missing Values

In the columns relevant to our analysis, only $\frac{768}{170.725} \approx 0.4\%$ of **value** entries we're missing. Let's explore where these are missing.

In [ ]:
missing_value = data[np.isnan(data['value'])==True]
missing_value['metric_item_label'].value_counts()

HIV/AIDS Prevalence             362
Police Killings                 300
Child Lead Levels 10+ mcg/dL     74
Population Density               12
Child Lead Testing               11
New Chlamydia Cases               3
Syphilis Prevalence               3
New Gonorrhea Cases               2
Child Lead Levels 5+ mcg/dL       1
Name: metric_item_label, dtype: int64

For now, we choose to drop these rows with missing values. If any of these labels turn out to show any importance in our later analysis, we will reconsider this decision.

In [ ]:
# Drop the NaNs
data = data.dropna()
print(data.isna().sum())

geo_label_city         0
geo_label_citystate    0
value                  0
date_label             0
metric_item_label      0
metric_subcat_label    0
dtype: int64


### 2.2. Data Exploration

How many cities do we have?

In [2]:
print(f"Total Cities in the dataset: {len(data.geo_label_city.unique())}")

NameError: name 'data' is not defined

Now we want to get the geographical coordinates for each city and put them in two dataset columns.

In [3]:
cities = list(data.geo_label_citystate.unique())

def get_lat_lng(city):
    geolocator = Nominatim(user_agent="city_explorer")
    location = geolocator.geocode(city)
    
    if location is not None:
        print(f"Location found for {city}: ({location.latitude}, {location.longitude})")
        return location.latitude, location.longitude
    else:
        print(f"Location not found for: {city}")
        return None

city_data = {'City': cities}
df = pd.DataFrame(city_data)

df['Lat_Lng'] = df['City'].apply(get_lat_lng)

NameError: name 'data' is not defined

In [4]:
df = df.dropna(subset=['Lat_Lng'])
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Lat_Lng'].tolist(), index=df.index)


NameError: name 'df' is not defined

In [5]:
us_map = folium.Map(location=[37.7749, -100.4194], zoom_start=5, width='90%', height='90%')

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=row['City'],
        fill=True,
        fill_opacity=0.7
    ).add_to(us_map)

us_map

NameError: name 'folium' is not defined

### 2.3. Zooming in on air quality

Let's first take a look at the Air Quality in different cities (*Warmer means worse Air Quality*).

In [6]:
# extract rows with air 1 and air 2
air_df = data_[data_["metric_item_label"].isin(["Poor Air Quality", "Hazardous Air Quality"])]
# extract rows with car, bike, bus, and walk
other_df = data_[data_["metric_item_label"].isin(["Drives Alone to Work", "Riding Bike to Work", "Public Transportation Use", "Walking to Work"])]

# merge the two dataframes on the first column
result_df = pd.merge(air_df, other_df, on=["geo_label_citystate", "date_label"])

# rename the columns
result_df = result_df.drop(columns=['metric_subcat_label_x','metric_subcat_label_y', 'geo_label_city_x', 'geo_label_city_y'])
result_df = result_df.rename(columns={"metric_item_label_x": "Air_quality_based_on", "metric_item_label_y": "Transportation", "value_x":"air_value", "value_y":"transport_value"})
result_df = result_df.drop_duplicates()
# convert date column to datetime format
result_df["date_label"] = pd.to_datetime(result_df["date_label"])

# sort the dataframe based on date
result_df = result_df.sort_values("date_label")
result_df

NameError: name 'data_' is not defined

In [7]:
#air_quality_data = data[data['metric_subcat_label'] == 'Air Quality']
result_df['year'] = result_df['date_label'].dt.year

# Create a new DataFrame containing city, latitude, longitude, and air quality values
air_quality_df = df.merge(result_df, left_on='City', right_on='geo_label_citystate')
air_quality_df = air_quality_df[['City', 'Latitude', 'Longitude', 'Air_quality_based_on', 'air_value', 'transport_value', 'date_label', 'Transportation', 'year']]

# Create a new DataFrame containing only the values computed using 'Poor Air Quality'
air_quality_df_poor = air_quality_df[air_quality_df['Air_quality_based_on']=='Poor Air Quality']

# Create a new DataFrame containing only the values computed using 'Hazardous Air Quality'
air_quality_df_haz = air_quality_df[air_quality_df['Air_quality_based_on']=='Hazardous Air Quality']

NameError: name 'result_df' is not defined

We have to take into account that there are two ways in which the Air Quality is computed:
- For one it is used the 'Poor Air Quality' value, which means that the numerator is the number of days when AQI was >50. 
- For the other one it is used the 'Hazardous Air Quality', which means that the numerator is the number of days when AQI was >100. 

In both cases the denominator is number of days for which AQI is recorded. 

Hence, lets first see if their distributions follow the same pattern or not. 

#### 2.3.1. Air Quality computed using "Poor Air Quality"

In [8]:
years = air_quality_df_poor['year'].unique()

# define subplot grid
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(15, 15), sharex=True, sharey=True)
plt.subplots_adjust(hspace=0.5)
fig.suptitle('Distribution of Air Quality computed using "Poor Air Quality"', fontsize=18, y=0.95)

# loop through tickers and axes
for year, ax in zip(years, axs.ravel()):
    # filter df for ticker and plot on specified axes
    air_quality_df_poor_y = air_quality_df_poor[air_quality_df_poor['year']==year]
    sns.histplot(data=air_quality_df_poor_y, x='air_value', bins=100, kde=True, ax=ax)

    # chart formatting
    ax.set_title(str(year))
    ax.set_xlabel("")

plt.show()

NameError: name 'air_quality_df_poor' is not defined

From the previous plot we can anticipate that the output of the study can be interesting since it shows the air quality distribution and even though there are lots of cities with air quality 0, there are also with high air quality. Furthermore, by looking at the year distribution we see that from the ones with low quality there has been a decrease in the last years and it has been an increase in the high quality ones.  

In [9]:
# Create the folium map and add the HeatMap layer
heatmap_data_poor = air_quality_df_poor[['Latitude', 'Longitude', 'air_value']].values.tolist()
HeatMap(heatmap_data_poor).add_to(us_map)
us_map

NameError: name 'air_quality_df_poor' is not defined

Overall, it seems that the west coast cities are the one with poorest air quality over the years. However, we need to look into the yearly data to see if this pattern changes during the years. (See section 5) 

#### 2.3.2. Air Quality computed using "Hazardous Air Quality"

In [10]:
years = air_quality_df_haz['year'].unique()

# define subplot grid
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(15, 15), sharex=True, sharey=True)
plt.subplots_adjust(hspace=0.5)
fig.suptitle('Distribution of Air Quality computed using "Hazardous Air Quality"', fontsize=18, y=0.95)

# loop through tickers and axes
for year, ax in zip(years, axs.ravel()):
    # filter df for ticker and plot on specified axes
    air_quality_df_haz_y = air_quality_df_haz[air_quality_df_haz['year']==year]
    sns.histplot(data=air_quality_df_haz_y, x='air_value', bins=100, kde=True, ax=ax)

    # chart formatting
    ax.set_title(str(year))
    ax.set_xlabel("")

plt.show()

NameError: name 'air_quality_df_haz' is not defined

In [11]:
# Create the folium map and add the HeatMap layer
heatmap_data_haz = air_quality_df_haz[['Latitude', 'Longitude', 'air_value']].values.tolist()
HeatMap(heatmap_data_haz).add_to(us_map)
us_map

NameError: name 'air_quality_df_haz' is not defined

FROM NOW ON WE'LL BE USING THE DATASET WHERE THE Air Quality IS computed using "Poor Air Quality", since both of them have the same distributions. 

#### 2.3.3. Time Series Plots 
<span style="color:red"> NOTE FROM LAURA: NOT SURE IF IT IS INSIGHTFUL THIS PART, IF IT IS, PLEASE COMMENT</span>

In [12]:
variables_of_interest = ['Public Transportation Use', 'Drives Alone to Work', 'Walking to Work', 'Riding Bike to Work']
filtered_data = air_quality_df_poor[air_quality_df_poor['Transportation'].isin(variables_of_interest)]
filtered_data.shape

NameError: name 'air_quality_df_poor' is not defined

In [13]:
filtered_data['Transportation'].unique()

NameError: name 'filtered_data' is not defined

In [14]:
sns.lineplot(data=air_quality_df_poor, x='year', y='air_value', hue='City', ci=None)
plt.title('Air Quality Over Time')
plt.xlabel('Year')
plt.ylabel('Air Quality Index')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

NameError: name 'sns' is not defined

**To get a less busy time series plot, we split the data into groups based on their first Air Quality Value (2010).**

In [15]:
# Get the first period air quality data
first_period_data = air_quality_df_poor[air_quality_df_poor['year'] == air_quality_df_poor['year'].min()]

# Define air quality groups
air_quality_groups = {'<30': [], '30-40': [], '40-60': [], '60+': []}

# Assign cities to their air quality groups
for city in first_period_data['City'].unique():
    city_value = first_period_data[first_period_data['City'] == city]['air_value'].values[0]
    if city_value < 25:
        air_quality_groups['<30'].append(city)
    elif city_value < 40:
        air_quality_groups['30-40'].append(city)
    elif city_value < 60:
        air_quality_groups['40-60'].append(city)
    else:
        air_quality_groups['60+'].append(city)

# Print the air quality groups and their cities
for group, cities in air_quality_groups.items():
    print(f"{group}: {', '.join(cities)}")

NameError: name 'air_quality_df_poor' is not defined

In [16]:
for group, cities in air_quality_groups.items():
    plt.figure(figsize=(12, 6))  # Set the figure size (width, height) in inches
    group_data = air_quality_df_poor[air_quality_df_poor['City'].isin(cities)]
    sns.lineplot(data=group_data, x='date_label', y='air_value', hue='City', ci=None)
    plt.title(f'Air Quality Over Time for Cities with Initial AQI {group}')
    plt.xlabel('Year')
    plt.ylabel('Air Quality Index')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

NameError: name 'air_quality_groups' is not defined

In [17]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='City', y='air_value', data=air_quality_df_poor)
plt.title('Distribution of air_quality_data by City')
plt.xlabel('City')
plt.ylabel('Value')
plt.xticks(rotation=90)
plt.show()

NameError: name 'plt' is not defined

EXAPLAIN VARIANCE

In [18]:
# Create a boxplot for each variable of interest, grouped by city
for variable in variables_of_interest:
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='City', y='transport_value', data=filtered_data[filtered_data['Transportation'] == variable])
    plt.title(f'Distribution of {variable} by City')
    plt.xlabel('City')
    plt.ylabel('Value')
    plt.xticks(rotation=90)
    plt.show()

NameError: name 'plt' is not defined

#### 2.3.2 Correlation Matrix between air quality and transportation means

<span style="color:red"> NOTE FROM LAURA: IS THIS CORRECTLY CODED?</span>

In [19]:
# Reshape the filtered dataset to have one row per city and year, with columns for each variable of interest
pivot_data = filtered_data.pivot_table(index=['City', 'date_label'], columns=['Transportation'], values='transport_value', aggfunc=np.mean).reset_index()

# Add the 'Air Quality' column to the pivot table
pivot_data['Air Quality'] = air_quality_df_poor['air_value']

# Calculate the correlation matrix for the variables of interest, including 'Air Quality'
variables_of_interest = ['Air Quality', 'Public Transportation Use', 'Drives Alone to Work', 'Walking to Work', 'Riding Bike to Work']
correlation_matrix = pivot_data[variables_of_interest].corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True)
plt.title('Correlation Matrix of Air Quality and Transportation Methods')
plt.show()

NameError: name 'filtered_data' is not defined

**There seems to be** some interesting correlations. 

> But before we lock in on our proposedly important independent features: **[Public Transportation Use, Drives Alone to Work, Walking to Work, Riding Bike to Work]**, we must try to identify other features that have relationships to our target feature **"Air Quality"**.

In [20]:
# Do feature selection with Air Quality as target



**What if we split the data into one dataset per. city?**

## 3. Data analysis
- Describe your data analysis and explain what you've learned about the dataset.
- If relevant, talk about your machine-learning.


>**Clustering**:

*See whether there are any similarities or groups* 

In [21]:
air_quality_df_poor

NameError: name 'air_quality_df_poor' is not defined

In [22]:
unique_values = list(air_quality_df_poor['Transportation'].unique())
# create a dictionary mapping each unique value to a corresponding integer value
value_map = {value: i*10 for i, value in enumerate(unique_values)}
air_quality_df_poor['Transport_int'] = air_quality_df_poor['Transportation'].map(value_map)

scaler = StandardScaler()
my_data = air_quality_df_poor[['air_value', 'transport_value']]
X = scaler.fit_transform(my_data.values)
wss = []
for i in range(1,11):
    cluster = KMeans(n_clusters=i,init='random',n_init=10, max_iter=300,tol=1e-04, random_state=0)
    cluster.fit(X)
    wss.append(cluster.inertia_)
plt.plot(range(1,11),wss,marker ='o')


NameError: name 'air_quality_df_poor' is not defined

In [23]:
cluster_final = KMeans(n_clusters=3)
pred = cluster_final.fit_predict(X)
air_quality_df_poor['predict'] = pred
air_quality_df_poor

NameError: name 'KMeans' is not defined

In [24]:
#plotting the results:
plt.scatter(air_quality_df_poor['air_value'] , air_quality_df_poor['transport_value'], c=pd.factorize(air_quality_df_poor['predict'])[0])
plt.legend()
plt.show()


NameError: name 'plt' is not defined

In [25]:

# Define the clustering algorithm
agg = AgglomerativeClustering(n_clusters=3)

# Fit the clustering algorithm to the data
agg.fit(X)

# Extract the labels assigned to each data point
labels = agg.labels_
# plot the resulting clusters
plt.scatter(air_quality_df_poor['air_value'] , air_quality_df_poor['transport_value'], c=labels)
plt.show()

NameError: name 'X' is not defined

In [26]:

# Define the clustering algorithm
gmm = GaussianMixture(n_components=3)

# Fit the clustering algorithm to the data
gmm.fit(X)

# Extract the labels assigned to each data point
labels = gmm.predict(X)
# plot the resulting clusters
plt.scatter(air_quality_df_poor['air_value'] , air_quality_df_poor['transport_value'], c=labels)
plt.show()

NameError: name 'X' is not defined

>**Regression**: **Predict poor air quality based on transportation methods**

*Check the 2-5 cities who's air quality has gone up/down the most and do further analysis on these* 

In [27]:


# Specify the number of classes you want
num_classes = 5

# Use pd.cut() to create bins for the continuous variable
bins = pd.cut(air_quality_df_poor['air_value'], num_classes, labels=False)

# Assign the bin labels to the Y variable
air_quality_df_poor['air_label'] = bins

# Select relevant columns
X = air_quality_df_poor[['transport_value', 'Transportation']]
y = air_quality_df_poor['air_label']


# Convert categorical variables to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['Transportation'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Fit a logistic regression model to the training data
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict air quality values for the test data
y_pred = model.predict(X_test)

# Evaluate the performance of the model
score = model.score(X_test, y_test)
print('Accuracy:', score)
# Calculate the F1 score for each class and average them using 'macro' averaging
f1_macro = f1_score(y_test, y_pred, average='macro')

# Print the F1 score
print("F1 score (macro-averaged):", f1_macro)

NameError: name 'pd' is not defined

## 4. Genre. Which genre of data story did you use?
- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?


## 5. Final visualization
- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?

## 6. Discussion. Think critically about your creation
- What went well?,
- What is still missing? What could be improved?, Why?


## 7. Contributions. Who did what?
You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
It is not OK simply to write "All group members contributed equally".


## 8. Make sure that you use references when they're needed and follow academic standards.